## Selector de K (método del codo)

Hasta ahora, para hacer el K_Means Clustering, hemos asignado el número de clusters (K) al azar.  Esto está bien para entender mejor el algoritmo pero debiera haber una forma automatizada de hacerlo...y lo hay.  Hay varias formas, pero utilizaremos el método del codo o "elbow".

## Importación de librerías relevantes

In [2]:
import pandas as pd
import plotly.express as px
from kneed import KneeLocator
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

## Generación de datos sintéticos

Para ver cómo es el metodo, necesitamos unos datos. 

Generamos algunos datos utilizando una función de "conveniencia" que provee sklearn **make_blobs()**.  Esta función utiliza estos parámetros:

* **n_samples**    el número total de observaciones que se desean generar.
* **centers**      el número de centroides a generar.
* **cluster_std**  la desviación standard.

**make_blobs()** devuelve una tupla de dos valores:

* Un arreglo NumPy bi-dimensional con los valores x - y para cada una de las observaciones.
* Un arreglo NumPy uni-dimensional con las etiquetas del cluster al que pertenece cada observación.

In [3]:
atributos, etiquetas = make_blobs(
    n_samples = 200,
    centers = 3,
    cluster_std = 2.75,
    random_state = 42
)

Verificación de los datos generados

In [4]:
atributos[:5]

array([[  9.77075874,   3.27621022],
       [ -9.71349666,  11.27451802],
       [ -6.91330582,  -9.34755911],
       [-10.86185913, -10.75063497],
       [ -8.50038027,  -4.54370383]])

In [5]:
etiquetas[:5]

array([1, 0, 2, 2, 2])

Es buena práctica estandarizar los datos

In [6]:
escalador = StandardScaler()
datos_escalados = escalador.fit_transform(atributos)
datos_escalados[:5]

array([[ 2.13082109,  0.25604351],
       [-1.52698523,  1.41036744],
       [-1.00130152, -1.56583175],
       [-1.74256891, -1.76832509],
       [-1.29924521, -0.87253446]])

## Iteraciones de K-Means

Cada vez que se ejecuta el método de K-Means, al terminar, calcula el valor de la suma de los errores cuadrados, o **SSE** (por sus siglas en inglés).  Para ejecutar el método "elbow", repetimos el K-Means varias veces, variando el valor de K, y registramos el SSE para cada K.

El instanciador de KMeans() puede manejar varios parámetros, entre ellos:

* **init**:  controla la técnica de inicialización.  El valor default es "random" pero se puede utilizar "k-means++" si se desea que converja más rápido
* **n-clusters**:  el número de clusters que deseamos
* **n_init**:  permite fijar cuantas veces se repite el proceso con cada valor de k.  Al terminar devuelve el valor más bajo de SSE que haya encontrado.  Valor default = 10
* **max_iter**:  el máximo número de iteraciones para cada K, si no ha habido convergencia
* **random_state**:  un valor semilla para que pueda ser reproducible el proceso

**NOTA:**

**SSE** es un término matemático.  En análisis de Clusters se conoce más como Within Cluster Sum of Squares (**WCSS**).  **SSE** y **WCSS** se utilizan intercambiablemente.

In [7]:
kmeans_kwargs = {
    "init": "random",
    "n_init": 10,
    "max_iter": 300,
    "random_state": 42,
}

# Creamos una lista para almacenar los valores de SSE
wcss = []
for k in range(1, 11):
    kmeans = KMeans(n_clusters = k, **kmeans_kwargs)
    kmeans.fit(datos_escalados)
    wcss.append(kmeans.inertia_)

## Ajustamos (entrenamos) con nuestro datos

In [8]:
kmeans.fit(datos_escalados)

KMeans(init='random', n_clusters=10, n_init=10, random_state=42)

## Estadísticas generadas por kmeans

Las estadísticas de la corrida, de las indicadadas en **n_init**, que haya generado el valor más bajo de WCSS, se pueden obtener como un atributo de kmeans, luego de correr el .fit. 

In [9]:
# El valor más bajo de WCSS
kmeans.inertia_

28.200015604968428

In [10]:
# las ubicaciones finales de los centroides
kmeans.cluster_centers_

array([[ 1.36880838,  0.36574906],
       [-1.25230326, -1.13288656],
       [-0.53282823, -1.24663692],
       [ 2.03875092, -0.07910488],
       [-0.07622608,  0.68009152],
       [-0.22595506,  1.51436618],
       [ 0.66466001,  0.18521947],
       [ 0.63531349,  1.23630902],
       [ 1.08979703, -0.34746472],
       [-0.75193401,  0.99069147]])

In [11]:
# El número de iteraciones que fueron necesarias para converger en esa corrida
kmeans.n_iter_

12

Finalmente, las asignaciones de clusters se almacenan en un arreglo NumPy uni-dimensional, en kmeans.labels_.

Si queremos ver las primeras 5 etiquetas:

In [12]:
kmeans.labels_[:5]

array([3, 9, 1, 1, 1], dtype=int32)

## Selección del número más adecuado de K

In [13]:
kmeans_kwargs = {
    "init": "random",
    "n_init": 10,
    "max_iter": 300,
    "random_state": 42,
}

# Creamos una lista para almacenar los valores de WCSS
wcss = []
for k in range(1, 11):
    kmeans = KMeans(n_clusters = k, **kmeans_kwargs)
    kmeans.fit(datos_escalados)
    wcss.append(kmeans.inertia_)

Veamos los WCSS

In [14]:
wcss

[400.0,
 173.23074893877768,
 74.57960106819853,
 61.372276288096515,
 52.27538725902111,
 45.18296492976122,
 40.4655696972595,
 34.79423830005605,
 30.74195582754622,
 28.200015604968428]

Vemos que van en orden descendente para cada valor de K ascendente.

Grafiquémos los WCSS vrs el valor de K

In [15]:
datos_WCSS = pd.DataFrame(range(1, 11), columns = ["K"])
datos_WCSS["WCSS"] = wcss
datos_WCSS

,K,WCSS
0,1,400.000000
1,2,173.230749
2,3,74.579601
3,4,61.372276
4,5,52.275387
5,6,45.182965
6,7,40.465570
7,8,34.794238
8,9,30.741956
9,10,28.200016


In [16]:
fig = px.line(datos_WCSS, x = "K" , y = "WCSS", title='WCSS vrs K')
fig.show()

En este caso es bastante fácil determinar cuál es el mejor valor de K.  Podría ser K = 2 pero aun hay bastante cambio entre K = 2 y K = 3.  Después de K = 3 el cambio es muy poco así que seleccionamos K = 3 como el número óptimo de clusters (obviamente...así diseñamos este conjunto de datos.

No siempre es tan obvio, y para eso podemos utilizar la librería **kneed** para determinarlo automáticamente.

In [17]:
localizador_codo = KneeLocator(range(1, 11), 
                               wcss, 
                               curve = "convex", 
                               direction = "decreasing"
                              )

localizador_codo.elbow

np.int64(3)

## Reconocimientos

Este tutorial es una adaptación de la guía que aparece en:  https://realpython.com/k-means-clustering-python/

**K-Means Clustering in Python: A Practical Guide**
by Kevin Arvai 

Traducción libre por Luis R. Furlán
Agosto de 2021